Dưới đây là phiên bản code hoàn chỉnh về việc cào dữ liệu từ web và quy trinh hoạt động của nó:
1. Khởi tạo môi trường và các thư viện cần thiết
2. Thiết lập thư mục lưu trữ:
    - Định nghĩa biến output_dir là đường dẫn đến thư mục lưu trữ dữ liệu đã crawl.
    - Kiểm tra và tạo thư mục output_dir nếu nó chưa tồn tại.
3. Khởi tạo trình duyệt Edge với Selenium:
    - Định nghĩa hàm get_driver() để khởi tạo trình duyệt Edge với đường dẫn đến msedgedriver.exe.
4. Hàm extract_and_save_house_data(page_number):
    - Hàm này nhận đầu vào là số trang page_number và sử dụng Selenium để điều hướng đến từng trang web bất động sản.
    - Thu thập mã nguồn HTML của trang bằng cách sử dụng driver.page_source.
    - Sử dụng BeautifulSoup để phân tích cú pháp HTML và trích xuất thông tin nhà từ các thẻ HTML tương ứng.
    - Dữ liệu cụ thể được trích xuất bao gồm: tiêu đề, giá, diện tích, số phòng ngủ, số phòng tắm, địa chỉ, mã tin, ngày đăng, và nội dung tin đăng.
    - Dữ liệu sau khi trích xuất được lưu vào các tệp văn bản .txt trong thư mục output_dir. Mỗi trang web sẽ có một thư mục con và mỗi bài viết sẽ có một tệp tin riêng.
    - Nếu có lỗi xảy ra trong quá trình xử lý, hàm sẽ bắt và in ra thông báo lỗi, sau đó sẽ thoát và đóng trình duyệt để tránh rò rỉ bộ nhớ.
5. Vòng lặp chính:
    - Sử dụng ThreadPoolExecutor để thực thi extract_and_save_house_data(page_number) cho mỗi trang từ 1 đến total_pages.
    - Với mỗi future được hoàn thành (as_completed), kiểm tra kết quả và in thông báo nếu có lỗi xảy ra trong quá trình xử lý một trang.

**Lợi ích:** Tự động hóa quy trình thu thập dữ liệu từ trang web, tiết kiệm thời gian và công sức so với việc thủ công. Thu thập thông tin chi tiết và cụ thể về các bất động sản như tiêu đề, giá cả, diện tích, và các thông tin khác. Tạo ra một cơ sở dữ liệu văn bản dễ dàng quản lý và phân tích.

**Hạn chế:** Việc cào dữ liệu này mất rất nhiều thời gian cần được giám sát liên tục đề phòng có lỗi xảy ra


In [2]:
import os
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from bs4 import BeautifulSoup

# Đường dẫn đến thư mục lưu trữ dữ liệu đã crawl
output_dir = "data_crawled_batdongsan"

# Tạo thư mục nếu chưa tồn tại
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Khởi tạo EdgeDriver mà không có tùy chọn nào.
def get_driver():
    edge_driver_path = "C:/EdgeDriver/msedgedriver.exe"
    service = EdgeService(executable_path=edge_driver_path)
    return webdriver.Edge(service=service)

# Hàm để thu thập dữ liệu từ một trang và lưu thông tin bài báo
def extract_and_save_house_data(page_number):
    driver = get_driver()
    try:
        print(f"Đang lấy dữ liệu từ trang {page_number}...")
        driver.get(f'https://batdongsan.vn/ban-nha-ho-chi-minh/p{page_number}')
        html_content = driver.page_source
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Tìm tất cả các mục nhà
        house_hrefs = []
        house_items = soup.find_all("div", class_="uk-grid uk-grid-small uk-grid-width-1-1")
        
        for house in house_items:
            name_divs = house.find_all("div", class_="name")
            for name_div in name_divs:
                a_tag = name_div.find("a", href=True)
                if a_tag:
                    house_hrefs.append(a_tag['href'])
        
        print(f"Tìm thấy {len(house_hrefs)} hrefs trên trang {page_number}.")
        
        for index, href in enumerate(house_hrefs):
            print(f"Đang xử lý href {index + 1}/{len(house_hrefs)}: {href}")
            driver.get(href)
            time.sleep(5)  # Dừng lại trong 5 giây để đảm bảo trang web đã được tải hoàn toàn.

            house_html_content = driver.page_source
            house_soup = BeautifulSoup(house_html_content, 'html.parser')

            # Extract the title
            title_element = house_soup.select_one('.project-global-object-block-003.information-custom .uk-panel-title span')
            title = title_element.get_text(strip=True) if title_element else "No Title"

            # Extract the price
            price_element = house_soup.select_one('.project-global-object-block-003.information-custom .price')
            price = price_element.get_text(strip=True) if price_element else "No Price"

            # Extract the specific details
            area, bedrooms, bathrooms, address, listing_id = "No Area", "No Bedrooms", "No Bathrooms", "No Address", "No Listing ID"
            details_list = house_soup.select('.project-global-object-block-003.information-custom .uk-list li')
            for detail in details_list:
                strong_tag = detail.find('strong')
                if strong_tag:
                    key = strong_tag.get_text(strip=True)
                    value = strong_tag.next_sibling.strip() if strong_tag.next_sibling else None
                    if not value:
                        value = strong_tag.find_next_sibling(string=True).strip() if strong_tag.find_next_sibling(string=True) else None
                    if "Diện tích" in key:
                        area = value
                    elif "Phòng ngủ" in key:
                        bedrooms = value
                    elif "Phòng WC" in key:
                        bathrooms = value
                    elif "Địa chỉ" in key:
                        address = value
                    elif "Mã tin" in key:
                        listing_id = value

            # Extract the datetime for "Ngày đăng:"
            date_element = house_soup.select_one('.project-global-object-block-003.information-custom .timeago')
            date = date_element['datetime'] if date_element else "No Date"

            # Locate the container for "Nội dung tin đăng"
            content_element = house_soup.select_one('.project-global-object-block-003.block-custom .content')
            # Extract the text content
            if content_element:
                content_text = content_element.get_text(separator=' ', strip=True)
            else:
                content_text = "No content found"
            
            # Tạo thư mục con cho số trang nếu nó chưa tồn tại.
            sub_dir = os.path.join(output_dir, str(page_number))
            if not os.path.exists(sub_dir):
                os.makedirs(sub_dir)
            
            # Define file name
            file_name = os.path.join(sub_dir, f"{index + 1}.txt")
            
            # Prepare file content
            file_content = (
                f"Title: {title}\n"
                f"Price: {price}\n"
                f"Area: {area}\n"
                f"Bedrooms: {bedrooms}\n"
                f"Bathrooms: {bathrooms}\n"
                f"Address: {address}\n"
                f"Listing ID: {listing_id}\n"
                f"Date: {date}\n"
                f"Content:\n{content_text}\n"
            )
            
            # Write file content to file
            with open(file_name, "w", encoding="utf-8") as file:
                file.write(file_content)
            
            print(f"Đã lưu dữ liệu từ bài viết {href} vào tệp {file_name}")
    except Exception as e:
        print(f"Có lỗi xảy ra khi xử lý trang {page_number}: {str(e)}")
    finally:
        driver.quit()

total_pages = 500  # điều chỉnh con số này dựa trên số lượng trang mà bạn muốn thu thập.
with ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(extract_and_save_house_data, page_number) for page_number in range(1, total_pages + 1)]
    for future in as_completed(futures):
        try:
            future.result()
        except Exception as e:
            print(f"Có lỗi xảy ra khi xử lý một trang: {str(e)}")

print("Quá trình thu thập dữ liệu đã hoàn tất.")

Đang lấy dữ liệu từ trang 3...
Tìm thấy 20 hrefs trên trang 3.
Đang xử lý href 1/20: https://batdongsan.vn/gap-ban-giam-13-ty-nha-quan-3-60m2-ngang-5m-chi-5x-ty-r284992
Đã lưu dữ liệu từ bài viết https://batdongsan.vn/gap-ban-giam-13-ty-nha-quan-3-60m2-ngang-5m-chi-5x-ty-r284992 vào tệp data_crawled_batdongsan\3\1.txt
Đang xử lý href 2/20: https://batdongsan.vn/ban-nha-go-vap-hxt-tranh-nguyen-van-khoi-p11-58m2-5-tang-chi-97-ty-r284982
Đã lưu dữ liệu từ bài viết https://batdongsan.vn/ban-nha-go-vap-hxt-tranh-nguyen-van-khoi-p11-58m2-5-tang-chi-97-ty-r284982 vào tệp data_crawled_batdongsan\3\2.txt
Đang xử lý href 3/20: https://batdongsan.vn/ban-nha-moi-4-tang-hxh-1-duong-tinh-lo-10-binh-tri-dong-binh-tan-48m2-nhinh-4ty-r284979
Đã lưu dữ liệu từ bài viết https://batdongsan.vn/ban-nha-moi-4-tang-hxh-1-duong-tinh-lo-10-binh-tri-dong-binh-tan-48m2-nhinh-4ty-r284979 vào tệp data_crawled_batdongsan\3\3.txt
Đang xử lý href 4/20: https://batdongsan.vn/ban-nha-mat-tien-duong-nhua-18m-an-phu-q2-16

Thu thập dữ liệu từ trang web batdongsan.vn và tổ chức thông tin chi tiết về bất động sản vào một tệp CSV để dễ dàng quản lý và phân tích.Tự động hóa quá trình thu thập và xử lý dữ liệu từ nhiều trang web bất động sản. Tạo ra một cơ sở dữ liệu dễ dàng truy cập và sử dụng để phân tích thị trường bất động sản. Tối ưu hóa thời gian và công sức so với việc thủ công thu thập và nhập liệu. 
- Khởi tạo một danh sách rỗng data để lưu thông tin của các căn nhà từ các tệp tin văn bản.
- Sử dụng os.listdir(output_dir) để lấy danh sách các tệp và thư mục trong output_dir
- Duyệt qua từng page_number trong danh sách các thư mục (được tạo ra cho mỗi trang web đã cào).
- Tạo đường dẫn đầy đủ tới mỗi tệp tin văn bản bằng cách kết hợp page_dir và file_name.
- Kiểm tra nếu tệp không phải là ẩn và có đuôi mở rộng là .txt, mở tệp và đọc nội dung.
- Mỗi tệp tin chứa thông tin về một căn nhà, được lưu theo định dạng cụ thể (ví dụ: Tiêu đề, Giá, Diện tích, Số phòng ngủ, ...).
- Sử dụng các chỉ số của dòng để trích xuất thông tin tương ứng và lưu vào một dictionary house_info.
- Mỗi house_info (dictionary) đại diện cho một bản ghi thông tin chi tiết của một căn nhà.
- Thêm house_info vào danh sách data để tạo thành một danh sách các dictionaries chứa thông tin của tất cả các căn nhà.

In [1]:
import os
import pandas as pd

# Thư mục chứa dữ liệu đã cào
output_dir = "data_crawled_batdongsan"

# Tạo danh sách để lưu trữ dữ liệu
data = []

# Duyệt qua tất cả các tệp đã lưu trong thư mục
for page_number in os.listdir(output_dir):
    page_dir = os.path.join(output_dir, page_number)
    if os.path.isdir(page_dir) and not page_number.startswith('.'):
        for file_name in os.listdir(page_dir):
            file_path = os.path.join(page_dir, file_name)
            if not file_name.startswith('.') and file_path.endswith('.txt'):
                try:
                    with open(file_path, "r", encoding="utf-8") as file:
                        lines = file.readlines()
                        # Khởi tạo dictionary để lưu trữ thông tin của mỗi căn nhà
                        house_info = {
                            "Title": lines[0].replace("Title: ", "").strip(),
                            "Price": lines[1].replace("Price: ", "").strip(),
                            "Area": lines[2].replace("Area: ", "").strip(),
                            "Bedrooms": lines[3].replace("Bedrooms: ", "").strip(),
                            "Bathrooms": lines[4].replace("Bathrooms: ", "").strip(),
                            "Address": lines[5].replace("Address: ", "").strip(),
                            "Listing ID": lines[6].replace("Listing ID: ", "").strip(),
                            "Date": lines[7].replace("Date: ", "").strip(),
                            "Content": "".join(lines[9:]).strip()  # Nội dung bắt đầu từ dòng thứ 10
                        }
                        data.append(house_info)
                except Exception as e:
                    print(f"Error reading file {file_path}: {e}")

# Tạo DataFrame từ danh sách dữ liệu
df = pd.DataFrame(data)

# Lưu DataFrame vào tệp CSV
csv_file_path = "raw_data.csv"
df.to_csv(csv_file_path, index=False, encoding="utf-8-sig")

print(f"Dữ liệu đã được lưu vào tệp {csv_file_path}")

Error reading file data_crawled_batdongsan\138\17.txt: list index out of range
Error reading file data_crawled_batdongsan\250\6.txt: list index out of range
Dữ liệu đã được lưu vào tệp raw_data.csv
